In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
MAX_LEN = 50

In [17]:
map_mods = [
    "Monsters reflect X% of Elemental Damage",
    "Monsters reflect X% of Physical Damage",
    "Players cannot Regenerate Life, Mana or Energy Shield",
    "Cannot Leech from Monsters",
    "Players are Cursed with Temporal Chains",
    "-X% maximum Player Resistances",
    "Players have X% less Recovery Rate of Life and Energy Shield",
    "Monsters are Hexproof",
    "Unique Bosses are Possessed",
    "Players have X% reduced effect of Non-Curse Auras from Skills",
    "Players are Cursed with Enfeeble",
    "Monsters have X% increased Critical Strike Chance",
    "Unique Boss deals X% increased Damage",
    "X% increased Monster Damage",
    "Monsters deal X% extra Physical Damage as Cold",
    "Monsters deal X% extra Physical Damage as Fire",
    "Monsters deal X% extra Physical Damage as Lightning",
    "Area has patches of Shocked Ground which increase Damage taken by X%",
    "All Monster Damage from Hits always Ignites",
    "Monsters cannot be Stunned",
    "X% increased Monster Movement Speed X% increased Monster Attack Speed X% increased Monster Cast Speed",
    "Monsters have a X% chance to cause Elemental Ailments on Hit",
    "Monsters Poison on Hit",
    "Monsters' Action Speed cannot be modified to below base value Monsters cannot be Taunted",
    "Monsters have X% chance to Impale with Attacks",
    "Monsters have X% increased Area of Effect",
    "Area has patches of Consecrated Ground",
    "Monsters gain X% of Maximum Life as Extra Maximum Energy Shield",
    "Unique Boss has X% increased Life Unique Boss has X% increased Area of Effect",
    "X% more Monster Life",
    "Players cannot inflict Exposure",
    "+X% Monster Physical Damage Reduction",
    "+X% Monster Chaos Resistance +X% Monster Elemental Resistance",
    "X% less effect of Curses on Monsters",
    "Monsters have +X% chance to Suppress Spell Damage",
    "Players have X% reduced Chance to Block Players have X% less Armour",
    "Players have -X% to amount of Suppressed Spell Damage Prevented Monsters have X% increased Accuracy Rating",
    "Monsters have X% chance to Avoid Elemental Ailments",
    "Monsters take X% reduced Extra Damage from Critical Strikes",
    "Buffs on Players expire X% faster",
    "Players gain X% reduced Flask Charges",
    "Players have X% less Area of Effect",
    "Players have X% less Cooldown Recovery Rate",
    "Players have X% less Accuracy Rating",
    "Players are Cursed with Vulnerability",
    "Players are Cursed with Elemental Weakness",
    "Monsters' skills Chain 2 additional times",
    "Monsters fire 2 additional Projectiles",
    "Monsters have a X% chance to avoid Poison, Impale, and Bleeding",
    "Monsters gain an Endurance Charge on Hit",
    "Monsters gain a Frenzy Charge on Hit",
    "Monsters gain a Power Charge on Hit",
    "Monsters Blind on Hit",
    "Monsters Maim on Hit with Attacks",
    "Monsters Hinder on Hit with Spells",
    "Monsters steal Power, Frenzy and Endurance charges on Hit",
    "Area has patches of Burning Ground",
    "Area has patches of Chilled Ground",
    "Area has patches of desecrated ground",
    "Area is inhabited by Abominations",
    "Area is inhabited by Animals",
    "Area is inhabited by Demons",
    "Area is inhabited by Ghosts",
    "Area is inhabited by Goatmen",
    "Area is inhabited by Humanoids",
    "Area is inhabited by Cultists of Kitava",
    "Area is inhabited by Lunaris fanatics",
    "Area is inhabited by ranged monsters",
    "Area is inhabited by Skeletons",
    "Area is inhabited by Solaris fanatics",
    "Area is inhabited by Sea Witches and their Spawn",
    "Area is inhabited by Undead",
    "Area has increased monster variety",
    "Area is inhabited by 2 additional Rogue Exiles",
    "Area contains many Totems",
    "Rare Monsters each have a Nemesis Mod X% more Rare Monsters",
    "Area contains two Unique Bosses",
    "Magic Monster Packs each have a Bloodline Mod X% more Magic Monsters",
    "Slaying Enemies close together has a X% chance to attract monsters from Beyond"
    ]

map_enchants = [
    "Map Boss is surrounded by Tormented Spirits",
    "Map has a Vaal Side Area",
    "Does not consume Sextant Uses",
    "Map has an additional random Modifier from Kirac's Crafting Bench",
    "Delirium Reward Type: Players in Area are 20% Delirious",
    "Area contains a Blight Encounter",
    "Area contains an additional Smuggler's Cache",
    "Areas contain Ritual Altars",
    "Area contains 3 additional Harbingers",
    "Area contains an additional Legion Encounter",
    "Area is haunted by 5 additional Tormented Spirits",
    "Area contains The Sacred Grove",
    "Area contains 2 additional Abysses",
    "Area contains 3 additional Breaches",
    "Area contains 3 additional Essences",
    "Area contains 4 additional Strongboxes",
    "Area contains Metamorph Monsters",
    "Contains an additional Expedition Encounter",
    "Area is inhabited by 10 additional Rogue Exiles", 
    "Area contains 10 additional guarded Vaal Vessels",
    "Area contains 5 additional Shrines",
    "Area is influenced by The Shaper",
    "Area is influenced by The Elder"
]

In [4]:
def match_substring_to_fullstring(substring, string_list):
    for string in string_list:
        if substring in string:
            return string
    return "String not found."

In [27]:
def map_mods_lower(map_mods):
    return [mod.lower() for mod in map_mods]
        
def populate_substring_list(map_mods, string_len):
    map_mods = map_mods_lower(map_mods)
    curr_substrings = []
    for mod in map_mods:
        curr_substring = ""
        lst = []
        i = 0
        while curr_substring != mod[-string_len:]:
            curr_substring = mod[i:i+string_len]
            lst.append(curr_substring)
            i += 1
        curr_substrings.append(lst)
    return curr_substrings

def generate_unique_substring(map_mods):
    mod_to_substring_dict = {}
    string_len = 3
    while len(mod_to_substring_dict) < len(map_mods):
        substring_lists = populate_substring_list(map_mods, string_len)

        num_substring_lists = len(substring_lists)
        for x in range(num_substring_lists):
            curr_substring_list = substring_lists[x]
            for substring in curr_substring_list:
                fullstring = match_substring_to_fullstring(substring, map_mods_lower(map_mods))
                if fullstring in mod_to_substring_dict:
                    continue
                substring_has_dupe = False
                for y in range(num_substring_lists): 
                    compare_substring_list = substring_lists[y]
                    if compare_substring_list == curr_substring_list:
                        continue
                    if substring in compare_substring_list:
                        substring_has_dupe = True
                        continue
                if substring_has_dupe == False:
                    # skip over 3 char substrings with space
                    if len(substring) == 3 and " " in substring:
                        continue
                    if "x%" in substring:
                        substring = substring.replace("x%", "\d+%")
                    mod_to_substring_dict[fullstring] = substring
                    break
        string_len += 1
    return mod_to_substring_dict

In [24]:
def generate_unique_substring_old(map_mods):
    mod_to_substring_dict = {}
    string_len = 3
    while len(mod_to_substring_dict) < len(map_mods):
        curr_substrings = []
        for mod in map_mods:
            curr_substring = ""
            i = 0
            while curr_substring != mod[-string_len:]:
                curr_substring = mod[i:i+string_len]
                curr_substrings.append(curr_substring)
                i += 1
        substring_index = 0
        num_substrings = len(curr_substrings)
        
        for x in range(num_substrings):
            substring = curr_substrings[x]
            substring_has_dupe = False
            for y in range(substring_index+1, num_substrings): 
                compare_substring = curr_substrings[y]
                print(substring, compare_substring)
                if substring == compare_substring:
                    substring_has_dupe = True
                    break
            if substring_has_dupe == False:
                print("added to big string")
                fullstring = match_substring_to_fullstring(substring, map_mods)
                print(fullstring, substring)
                mod_to_substring_dict[fullstring] = substring
                break
            substring_index += 1
        string_len += 1
    print(curr_substrings)
    return mod_to_substring_dict

In [25]:
map_mods_full = map_mods + map_enchants

In [28]:
def print_substrings_ordered(map_mods, unique_substring_dict):
    for mod in map_mods:
        print(mod + ": '" + unique_substring_dict[mod.lower()] + "'")

print_substrings_ordered(map_mods_full, generate_unique_substring(map_mods_full))

Monsters reflect X% of Elemental Damage: 'f el'
Monsters reflect X% of Physical Damage: 'of p'
Players cannot Regenerate Life, Mana or Energy Shield: 'reg'
Cannot Leech from Monsters: 'eec'
Players are Cursed with Temporal Chains: 'emp'
-X% maximum Player Resistances: '% ma'
Players have X% less Recovery Rate of Life and Energy Shield: 'ss r'
Monsters are Hexproof: 'hex'
Unique Bosses are Possessed: 're p'
Players have X% reduced effect of Non-Curse Auras from Skills: 'non'
Players are Cursed with Enfeeble: 'enf'
Monsters have X% increased Critical Strike Chance: 'd cr'
Unique Boss deals X% increased Damage: 'ss d'
X% increased Monster Damage: 'd monster d'
Monsters deal X% extra Physical Damage as Cold: 'col'
Monsters deal X% extra Physical Damage as Fire: ' as f'
Monsters deal X% extra Physical Damage as Lightning: 'htn'
Area has patches of Shocked Ground which increase Damage taken by X%: 'sho'
All Monster Damage from Hits always Ignites: 'all'
Monsters cannot be Stunned: 'stu'
X% i

In [33]:
quant_regex = {
    60: "Q.*([6-9]\d|\d{3})",
    70: "Q.*([7-9]\d|\d{3})",
    80: "Q.*([89]\d|\d{3})",
    90: "Q.*(9\d|\d{3})",
    100: "Q.*(\d{3})"
}

packsize_regex = {
    20: "S.*([2-9]\d)", 
    30: "S.*([3-9]\d)", 
    40: "S.*([4-9]\d)", 
    50: "S.*([5-9]\d)", 
    60: "S.*([6-9]\d)",
}

In [34]:
quant_regex[60]

'Q.*([6-9]\\d|\\d{3})'

In [36]:
test_map = """
    Item Class: Maps
    Rarity: Rare
    Grim Depths
    Marshes Map
    --------
    Map Tier: 16
    Item Quantity: +115% (augmented)
    Item Rarity: +56% (augmented)
    Monster Pack Size: +37% (augmented)
    Quality: +20% (augmented)
    --------
    Item Level: 83
    --------
    Area is inhabited by 2 additional Rogue Exiles
    Players cannot inflict Exposure
    -10% maximum Player Resistances
    Monsters reflect 18% of Elemental Damage
    Monsters are Hexproof
    Area has patches of Consecrated Ground
    Monsters cannot be Taunted
    Monsters' Action Speed cannot be modified to below base value
    Monsters Hinder on Hit with Spells
    --------
    Travel to this Map by using it in a personal Map Device. Maps can only be used once.
    --------
    Corrupted
"""